In [8]:
import pandas as pd 
import datetime
import numpy as np 
import matplotlib.pyplot as plt 
import statsmodels.api as sm
from pylab import rcParams
import warnings
from pandas.core.nanops import nanmean as pd_nanmean
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
sputnik_data = pd.read_csv('train.csv', sep =',')
sputnik_data.head(10)

,id,epoch,sat_id,x,y,z,x_sim,y_sim,z_sim,type
0,0,2014-01-01T00:00:00.000,0,-8855.823863,13117.780146,-20728.353233,-8843.131454,13138.221690,-20741.615306,train
1,1,2014-01-01T00:46:43.000,0,-10567.672384,1619.746066,-24451.813271,-10555.500066,1649.289367,-24473.089556,train
2,2,2014-01-01T01:33:26.001,0,-10578.684043,-10180.467460,-24238.280949,-10571.858472,-10145.939908,-24271.169776,train
3,3,2014-01-01T02:20:09.001,0,-9148.251857,-20651.437460,-20720.381279,-9149.620794,-20618.200201,-20765.019094,train
4,4,2014-01-01T03:06:52.002,0,-6719.092336,-28929.061629,-14938.907967,-6729.358857,-28902.271436,-14992.399986,train
5,5,2014-01-01T03:53:35.002,0,-3708.453525,-34767.115528,-7863.224747,-3726.986435,-34749.558551,-7921.459045,train
6,6,2014-01-01T04:40:18.003,0,-437.699227,-38249.612548,-234.351187,-463.278088,-38241.966025,-293.331552,train
7,7,2014-01-01T05:27:01.003,0,2863.147037,-39594.503233,7420.538280,2831.900642,-39595.997138,7364.088245,train
8,8,2014-01-01T06:13:44.004,0,6031.593902,-39056.319613,14731.102545,5996.014434,-39065.326088,14679.572942,train
9,9,2014-01-01T07:00:27.004,0,8950.655291,-36886.362968,21432.111677,8911.952800,-36900.814799,21387.028371,train


In [5]:
import numpy as np

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [3]:
x_best = []
y_best = []
z_best = []

In [9]:
err1 = 0
err2 = 0
err3 = 0
df_res = pd.DataFrame(columns=['id','error'])
for i in range(600):
#     print(i)
    df = sputnik_data[sputnik_data['sat_id']==i]
    k = len(df)-df['x'].isnull().sum()
    train = df[:int(k*2/3)]
    val = df[int(k*2/3):k]
    train['label'] = 'train'
    val['label'] = 'val'
    df1 = pd.concat((train,val), axis = 0)
    df1['target'] = np.where(df1.label == 'train', df1.x, np.nan)
    df2 = pd.concat((train,val), axis = 0)
    df2['target'] = np.where(df2.label == 'train', df2.y, np.nan)
    df3 = pd.concat((train,val), axis = 0)
    df3['target'] = np.where(df3.label == 'train', df3.z, np.nan)
    l = 24
    l1 = 21
    if i==265 or i==277:
        l=20
    if i==252 or 301:
        l=20
    if i==26 or i==199 or i==265 or i==277:
        l1=0
    if i==252 or 301:
        l1=0
#     if i in x_best:
    fit1 = ExponentialSmoothing(np.asarray(train['x']),seasonal_periods=24 , seasonal='add').fit()
    forecast1_1 = pd.Series(fit1.forecast(len(val)))
    forecast1_1.index = val.index
#     if i in y_best:
    fit2 = ExponentialSmoothing(np.asarray(train['y']),seasonal_periods=24 , seasonal='add').fit()
    forecast2_2 = pd.Series(fit1.forecast(len(val)))
    forecast2_2.index = val.index
#     if i in z_best:
    fit3 = ExponentialSmoothing(np.asarray(train['z']),seasonal_periods=24 , seasonal='add').fit()
    forecast3_3 = pd.Series(fit1.forecast(len(val)))
    forecast3_3.index = val.index
    lag_period = len(val)
    features1 = []
    for period_mult in range(1,2,1):
        df1["lag_period_{}".format(period_mult)] = df1.target.shift(period_mult*lag_period)
        features1.append("lag_period_{}".format(period_mult))
    lag_period1 = 24
    for period_mult in range(1,l,1):
        df1["lag_period_{}".format(period_mult+3)] = df1.target.shift(lag_period+period_mult)
        features1.append("lag_period_{}".format(period_mult+3))
    for period_mult in range(1,1,1):
        df1["lag_period_{}".format(period_mult+30)] = df1.target.shift(lag_period*2+period_mult+24)
        features1.append("lag_period_{}".format(period_mult+30))
    for period_mult in range(1,1,1):
        df1["lag_period_{}".format(period_mult+30)] = df1.target.shift(period_mult*lag_period+period_mult)
        features1.append("lag_period_{}".format(period_mult+30))
#     features1 = []

#     for period_mult in range(1,1,1):
#         df1["lag_period_{}".format(period_mult+3)] = 0
#         for i in train.index:
#     #         print(df1.index)
#             df1["lag_period_{}".format(period_mult+3)][i] = df1.target.shift(24*period_mult+i%24)[i]
#     #         print(df1.target.shift(24*period_mult)[i])
#         for i in val.index:
#             df1["lag_period_{}".format(period_mult+3)][i] = df1.target.shift(24*(period_mult+((i-train.index.max())//24))+(i%24))[i]
# #             print(df1.target.shift(24*period_mult*((i-len(train)+24)//24))[i])
#     #     df1.target.shift(period_mult*lag_period)
#         features1.append("lag_period_{}".format(period_mult+3))
        
    features2 = []
    for period_mult in range(1,2,1):
        df2["lag_period_{}".format(period_mult)] = df2.target.shift(period_mult*lag_period)
        features2.append("lag_period_{}".format(period_mult))
    for period_mult in range(1,l,1):
        df2["lag_period_{}".format(period_mult+3)] = df2.target.shift(lag_period+period_mult)
        features2.append("lag_period_{}".format(period_mult+3))
    for period_mult in range(1,1,1):
        df2["lag_period_{}".format(period_mult+30)] = df2.target.shift(lag_period*2+period_mult+24)
        features2.append("lag_period_{}".format(period_mult+30))
    for period_mult in range(1,1,1):
        df2["lag_period_{}".format(period_mult+30)] = df2.target.shift(period_mult*lag_period+period_mult)
        features2.append("lag_period_{}".format(period_mult+30))
    lag_period1 = 24
#     features1 = []
    for period_mult in range(1,1,1):
        df2["lag_period_{}".format(period_mult+3)] = 0
        for i in train.index:
    #         print(df1.index)
            df2["lag_period_{}".format(period_mult+3)][i] = df2.target.shift(24*period_mult+i%24)[i]
    #         print(df1.target.shift(24*period_mult)[i])
        for i in val.index:
            df2["lag_period_{}".format(period_mult+3)][i] = df2.target.shift(24*(period_mult+((i-train.index.max())//24))+(i%24))[i]
#             print(df2.target.shift(24*period_mult*((i-len(train)+24)//24))[i])
    #     df1.target.shift(period_mult*lag_period)
        features2.append("lag_period_{}".format(period_mult+3))
    
    
    features3 = []
    for period_mult in range(1,2,1):
        df3["lag_period_{}".format(period_mult)] = df3.target.shift(period_mult*lag_period)
        features3.append("lag_period_{}".format(period_mult))
    lag_period1 = 24
    for period_mult in range(1,l,1):
        df3["lag_period_{}".format(period_mult+3)] = df3.target.shift(lag_period+period_mult)
        features3.append("lag_period_{}".format(period_mult+3))
    for period_mult in range(1,1,1):
        df3["lag_period_{}".format(period_mult+30)] = df3.target.shift(period_mult*lag_period+period_mult)
        features3.append("lag_period_{}".format(period_mult+30))
    for period_mult in range(1,1,1):
        df3["lag_period_{}".format(period_mult+30)] = df3.target.shift(lag_period*2+period_mult+24)
        features3.append("lag_period_{}".format(period_mult+30))
    
    for period_mult in range(1,1,1):
        df3["lag_period_{}".format(period_mult+3)] = 0
        for i in train.index:
    #         print(df1.index)
            df3["lag_period_{}".format(period_mult+3)][i] = df3.target.shift(24*period_mult+i%24)[i]
    #         print(df1.target.shift(24*period_mult)[i])
        for i in val.index:
            df3["lag_period_{}".format(period_mult+3)][i] = df3.target.shift(24*(period_mult+((i-train.index.max())//24))+(i%24))[i]
#             print(df3.target.shift(24*period_mult*((i-len(train)+24)//24))[i])
    #     df1.target.shift(period_mult*lag_period)
        features3.append("lag_period_{}".format(period_mult+3))
#         features1.append("lag_period_{}".format(period_mult+3))
#     for period_mult in range(3,40,1):
#         df3["lag_period_{}".format(period_mult)] = df3.target.shift(int(k*2/3)-period_mult)
#         features3.append("lag_period_{}".format(period_mult))
#     for period_mult in range(3,40,1):
#         df1["lag_period_{}".format(period_mult)] = df1.target.shift(int(k*2/3)-period_mult)
#         features1.append("lag_period_{}".format(period_mult))
#     for period_mult in range(3,40,1):
#         df2["lag_period_{}".format(period_mult)] = df2.target.shift(int(k*2/3)-period_mult)
#         features2.append("lag_period_{}".format(period_mult))
    # # лаговые статистики
    df1['lagf_mean'] = df1[features1[:1]].mean(axis = 1)
    df2['lagf_mean'] = df2[features2[:1]].mean(axis = 1)
    df3['lagf_mean'] = df3[features3[:1]].mean(axis = 1)
    df1['lagf_mean1'] = df1[features1[1:]].mean(axis = 1)
    df2['lagf_mean1'] = df2[features2[1:]].mean(axis = 1)
    df3['lagf_mean1'] = df3[features3[1:]].mean(axis = 1)

    features1.extend(['lagf_mean'])
    features1.extend(['lagf_mean1'])
    model1 = LinearRegression()
    train_df1 = df1[df1.label == 'train'][features1 + ['target']].dropna()
    test_df1 = df1[df1.label == 'val'][features1]

    features2.extend(['lagf_mean'])
    features2.extend(['lagf_mean1'])
    model2 = LinearRegression()
    train_df2 = df2[df2.label == 'train'][features2 + ['target']].dropna()
    test_df2 = df2[df2.label == 'val'][features2]

    features3.extend(['lagf_mean'])
    features3.extend(['lagf_mean1'])
    model3 = LinearRegression()
    train_df3 = df3[df3.label == 'train'][features3 + ['target']].dropna()
    test_df3 = df3[df3.label == 'val'][features3]
#     print(test_df3.head())
    model1.fit(train_df1.drop('target', axis = 1) ,train_df1['target'])
#     if i not in x_best:
    forecast1 = model1.predict(test_df1)
    test_df1['prediction'] = forecast1
    model2.fit(train_df2.drop('target', axis = 1) ,train_df2['target'])
#     if i not in y_best:
    forecast2 = model2.predict(test_df2)
    test_df2['prediction'] = forecast2
    model3.fit(train_df3.drop('target', axis = 1) ,train_df3['target'])
#     if i not in z_best:
    forecast3 = model3.predict(test_df3)
    test_df1['prediction'] = forecast1
    test_df2['prediction'] = forecast2
    test_df3['prediction'] = forecast3
    err11 = 0
    err22 = 0
    err33 = 0
#     if i not in x_best:
#         err11 = smape(test_df1.prediction,val['x'])
#     else:
#         err11 = smape(forecast1_1,val['x'])
#     if i not in y_best:
#         err22 = smape(test_df2.prediction,val['y'])
#     else:
#         err22 = smape(forecast2_2,val['y'])
#     if i not in z_best:
#         err33 = smape(test_df3.prediction,val['z'])
#     else:
#         err33 = smape(forecast3_3,val['z'])
    err11 = smape(test_df1.prediction,val['x'])
    err22 = smape(test_df2.prediction,val['y'])
    err33 = smape(test_df3.prediction,val['z'])
#     print(smape(forecast1_1,val['x']))
#     print(smape(forecast2_2,val['y']))
#     print(smape(forecast3_3,val['z']))
    if(err11>smape(forecast1_1,val['x'])):
        err11 = smape(forecast1_1,val['x'])
        x_best.append(i)
    if(err22>smape(forecast2_2,val['y'])):
        err22 = smape(forecast2_2,val['y'])
        y_best.append(i)
    if(err33>smape(forecast3_3,val['z'])):
        err33 = smape(forecast3_3,val['z'])
        z_best.append(i)
        
    
    err1 += err11
    err2 += err22
    err3 += err33
    print(i,err11,'err1',err22,'err2',err33,'err3')
#     a1 =  np.hstack((np.array(df1.x[:k]),np.array(test_df1.prediction.values)))
#     a2 =  np.hstack((np.array(df2.y[:k]),np.array(test_df2.prediction.values)))
#     a3 =  np.hstack((np.array(df3.z[:k]),np.array(test_df3.prediction.values)))
#     a1 = pd.DataFrame(a1)
#     a2= pd.DataFrame(a2)
#     a3= pd.DataFrame(a3)
#     a1.index = df1.index
#     a2.index = df2.index
#     a3.index = df3.index
#     df['x'] = a1
#     df['y'] = a2
#     df['z'] = a3
#     df['error']  = np.linalg.norm(df[['x', 'y', 'z']].values - df[['x_sim', 'y_sim', 'z_sim']].values, axis=1)
#     df_cur = pd.DataFrame(df[k:])
#     df_cur = df_cur[['id','error']]
#     df_res = df_res.merge(df_cur, how='outer')

0 49.70399867287987 err1 53.64283163674844 err2 52.38189879274761 err3
1 7.637193576890376 err1 29.0093748236499 err2 29.056657589034927 err3
2 31.59733008667005 err1 30.37629127288946 err2 39.2361881147654 err3
3 23.973475577689182 err1 20.99790733247676 err2 25.873840600987513 err3
4 15.543608272671076 err1 19.073876941672392 err2 14.761250170426502 err3
5 0.1910318845991156 err1 0.012936905482234577 err2 0.0674059689205699 err3
6 8.480029852853647 err1 9.775155281095968 err2 7.974852761364627 err3
7 23.985548324858605 err1 32.90877118457516 err2 29.154822157054678 err3
8 0.20548980320629567 err1 4.466436858267978 err2 5.606924671517078 err3
9 0.5095212626366483 err1 2.9320138890701704 err2 1.1159848557193026 err3
10 20.001907764987543 err1 25.29740335252066 err2 25.73942800731473 err3
11 32.05130825567222 err1 29.73096591981868 err2 30.701787060428252 err3
12 27.683874455721295 err1 29.76034264059219 err2 27.912009217085654 err3
13 24.438205580397028 err1 29.642252578475773 err2 26.

112 2.5939586259799094 err1 1.6598216905424783 err2 1.3095143890509702 err3
113 3.734091471811396 err1 3.660796937059179 err2 3.1794066325694508 err3
114 2.6067509513365392 err1 1.6413129113147515 err2 1.8156103466114937 err3
115 2.794938960176571 err1 0.9755953685552297 err2 2.215923176158356 err3
116 0.6774379891333925 err1 0.6275863568492122 err2 0.21036170451186775 err3
117 1.429221625399423 err1 2.001001696332057 err2 3.17807739633489 err3
118 1.8737530223733287 err1 26.174186512277622 err2 24.351176476178942 err3
119 0.23530967461368169 err1 0.22618265279515481 err2 0.303010815496966 err3
120 0.6517483738844811 err1 1.1073696886097228 err2 4.422661534184458 err3
121 0.0871276941311967 err1 0.13011931513571456 err2 0.08021757671171754 err3
122 8.678229619392104 err1 8.891793543721521 err2 10.634775100175473 err3
123 21.95749960404605 err1 16.554039495936625 err2 17.364149386445018 err3
124 18.74790802289373 err1 27.003190998658717 err2 26.00691236904765 err3
125 1.934095841068727 

221 0.48784461847305305 err1 0.6582875595699673 err2 0.517057451544605 err3
222 1.0568799252521555 err1 1.0148075927248503 err2 0.9112523939007668 err3
223 43.77332397411741 err1 44.91092934239336 err2 46.03292559075238 err3
224 22.935858573154825 err1 42.63412736865201 err2 37.506758119137686 err3
225 4.596539514001009 err1 4.107944341438153 err2 2.7606369358310543 err3
226 0.3273095267849113 err1 0.11932510850190348 err2 0.2134636105871671 err3
227 0.43416440728188627 err1 0.8596257630462707 err2 0.5093013089577914 err3
228 0.7931905756996919 err1 17.30532273389485 err2 11.566209412485131 err3
229 1.638257560114267 err1 6.253114462606211 err2 0.4141135637104334 err3
230 8.69567916579457 err1 5.398085324371754 err2 9.465004228916056 err3
231 1.0684783802245583 err1 3.0304535420210446 err2 3.514090151064287 err3
232 2.3369846458390713 err1 2.5132064894141637 err2 2.2190202599096978 err3
233 0.07237133545510037 err1 0.42673553518441243 err2 0.08164299499749753 err3
234 0.077910839594786

331 0.7832838288631794 err1 0.06065663211698143 err2 0.6298443735472803 err3
332 0.13318329141723564 err1 0.26094698098854785 err2 2.1843654941452435 err3
333 4.4951564208742045 err1 7.025713853845323 err2 4.767708482628177 err3
334 28.482081887896875 err1 26.230968948006357 err2 26.59158221507035 err3
335 0.20598344333748494 err1 0.5165357533601808 err2 0.18209941980731398 err3
336 5.475987153894904 err1 5.8598767211733715 err2 3.879776389240976 err3
337 0.024618702130699097 err1 0.01632476174595013 err2 0.03906150444532148 err3
338 3.3577860808069677 err1 1.9171793319636805 err2 1.3755857381675347 err3
339 17.268768899282385 err1 17.090644077848168 err2 20.793327314058427 err3
340 48.39230454025809 err1 47.861132299321056 err2 37.37410536190669 err3
341 0.5735494763110764 err1 0.9310998681704992 err2 2.138443040927499 err3
342 0.04507932520728051 err1 0.07801569060315372 err2 0.04224604669903353 err3
343 0.1281189971690147 err1 0.335202453160336 err2 0.11935117875327936 err3
344 0.12

440 31.693074004293393 err1 32.76803024898569 err2 30.663067997472545 err3
441 48.56059611232565 err1 43.291986628063505 err2 88.44979442717718 err3
442 6.785620446694999 err1 9.65091394054118 err2 10.017926270527997 err3
443 21.08634205297419 err1 20.81329135740402 err2 25.5281921136078 err3
444 0.7207204166640221 err1 1.4848390958896112 err2 0.7341848463569465 err3
445 2.1829587547962657 err1 4.945304724986863 err2 3.225870281552175 err3
446 0.4626309445982872 err1 0.21265496909123527 err2 0.523057551130998 err3
447 8.05042325034516 err1 13.059315553461671 err2 11.495880686185084 err3
448 9.406806844585358 err1 3.463592887585835 err2 2.4891167470365287 err3
449 0.43684700663740256 err1 0.4401980763607423 err2 0.37947618924536614 err3
450 44.603313433669 err1 61.31115552649835 err2 25.882046626293928 err3
451 1.1777797486650403 err1 1.377273075225726 err2 1.2954458240554125 err3
452 7.123253313997357 err1 29.583481609615923 err2 31.691564915900255 err3
453 2.7579511089844595 err1 2.42

551 0.8496356933082778 err1 1.2128167304375865 err2 1.8644993164781032 err3
552 22.84743576834782 err1 20.261349963292588 err2 31.33467997323948 err3
553 1.165580888977168 err1 2.877633227726781 err2 0.06798432899240893 err3
554 11.178951434461295 err1 59.56201416313719 err2 46.31208745754515 err3
555 24.19091824918854 err1 29.72667814525263 err2 27.984347846018782 err3
556 12.089227091122934 err1 3.65520182967626 err2 17.93492022698408 err3
557 0.20887015492455943 err1 0.2675164775794882 err2 0.2327364365332905 err3
558 55.679994765793545 err1 54.43964745228701 err2 56.63302912412332 err3
559 29.939098914646287 err1 26.129216991940773 err2 27.55765916878875 err3
560 33.79635199192362 err1 31.966863366858625 err2 32.488978627478986 err3
561 2.041011488971088 err1 1.634059663746602 err2 1.3101563640600866 err3
562 0.14287932073405318 err1 1.1498305821856387 err2 0.256602204145425 err3
563 4.623199728915376 err1 8.642465564504315 err2 11.105775626926839 err3
564 0.4974343648739634 err1 1

In [10]:
err1/600

13.049336425199742

In [11]:
err2/600

15.100354059670018

In [12]:
err3/600

15.757624629424866

In [13]:
x_best

[1,
 10,
 17,
 21,
 25,
 26,
 27,
 39,
 43,
 48,
 50,
 52,
 55,
 56,
 66,
 70,
 72,
 74,
 76,
 77,
 78,
 79,
 80,
 84,
 86,
 87,
 89,
 95,
 96,
 99,
 102,
 103,
 105,
 107,
 117,
 118,
 124,
 126,
 128,
 131,
 146,
 147,
 149,
 150,
 153,
 163,
 165,
 166,
 174,
 177,
 178,
 180,
 182,
 185,
 187,
 189,
 190,
 192,
 193,
 199,
 201,
 202,
 204,
 205,
 206,
 214,
 219,
 220,
 224,
 228,
 236,
 238,
 243,
 245,
 253,
 255,
 260,
 265,
 267,
 269,
 270,
 275,
 278,
 279,
 282,
 283,
 285,
 286,
 289,
 293,
 301,
 324,
 325,
 345,
 348,
 353,
 354,
 361,
 363,
 366,
 370,
 371,
 373,
 384,
 391,
 392,
 402,
 406,
 407,
 412,
 418,
 421,
 422,
 424,
 426,
 432,
 441,
 443,
 450,
 452,
 455,
 457,
 460,
 461,
 464,
 470,
 473,
 478,
 480,
 482,
 483,
 485,
 486,
 487,
 491,
 492,
 493,
 496,
 497,
 502,
 503,
 506,
 507,
 511,
 515,
 518,
 523,
 524,
 527,
 529,
 535,
 543,
 550,
 551,
 552,
 554,
 555,
 558,
 561,
 569,
 577,
 591,
 598]

In [14]:
df_res = pd.DataFrame(columns=['id','error'])
for i in range(600):
#     if(i%10==0):
    print(i)
    df = sputnik_data[sputnik_data['sat_id']==i]
    k = len(df)-df['x'].isnull().sum()
    train = df[:k]
    val = df[k:]
    train['label'] = 'train'
    val['label'] = 'val'
    df1 = pd.concat((train,val), axis = 0)
    df1['target'] = np.where(df1.label == 'train', df1.x, np.nan)
    df2 = pd.concat((train,val), axis = 0)
    df2['target'] = np.where(df2.label == 'train', df2.y, np.nan)
    df3 = pd.concat((train,val), axis = 0)
    df3['target'] = np.where(df3.label == 'train', df3.z, np.nan)
    lag_period = len(val)
    features1 = []
    l = 24
    p = 5
    if i==26 or i==199 or i==265 or i==277:
        p=4
    if i==252 or 301:
        p=3
        
        
    if i in x_best:
        fit1 = ExponentialSmoothing(np.asarray(train['x']),seasonal_periods=24 , seasonal='add').fit()
        forecast1_1 = pd.Series(fit1.forecast(len(val)))
        forecast1_1.index = val.index
    if i in y_best:
        fit2 = ExponentialSmoothing(np.asarray(train['y']),seasonal_periods=24 , seasonal='add').fit()
        forecast2_2 = pd.Series(fit1.forecast(len(val)))
        forecast2_2.index = val.index
    if i in z_best:
        fit3 = ExponentialSmoothing(np.asarray(train['z']),seasonal_periods=24 , seasonal='add').fit()
        forecast3_3 = pd.Series(fit1.forecast(len(val)))
        forecast3_3.index = val.index    
        
    
    for period_mult in range(1,3,1):
        df1["lag_period_{}".format(period_mult)] = df1.target.shift(period_mult*lag_period)
        features1.append("lag_period_{}".format(period_mult))
    for period_mult in range(1,l,1):
        df1["lag_period_{}".format(period_mult+3)] = df1.target.shift(lag_period+period_mult)
        features1.append("lag_period_{}".format(period_mult+3))
#     for period_mult in range(1,3,1):
#         df1["lag_period_{}".format(period_mult+3)] = df1.target.shift(period_mult*lag_period+24)
#         features1.append("lag_period_{}".format(period_mult+3))
# #     for period_mult in range(1,p,1):
#         df1["lag_period_{}".format(period_mult+3)] = 0
#         for i in train.index:
#             df1["lag_period_{}".format(period_mult+3)][i] = df1.target.shift(24*period_mult+i%24)[i]
#         for i in val.index:
#             df1["lag_period_{}".format(period_mult+3)][i] = df1.target.shift(24*(period_mult+((i-train.index.max())//24))+(i%24))[i]
#         features1.append("lag_period_{}".format(period_mult+3))
        
        
    features2 = []
    for period_mult in range(1,3,1):
        df2["lag_period_{}".format(period_mult)] = df2.target.shift(period_mult*lag_period)
        features2.append("lag_period_{}".format(period_mult))
    for period_mult in range(1,l,1):
        df2["lag_period_{}".format(period_mult+3)] = df2.target.shift(lag_period+period_mult)
        features2.append("lag_period_{}".format(period_mult+3))
#     for period_mult in range(1,3,1):
#         df2["lag_period_{}".format(period_mult+3)] = df2.target.shift(period_mult*lag_period+24)
#         features2.append("lag_period_{}".format(period_mult+3))    
#     for period_mult in range(1,p,1):
#         df2["lag_period_{}".format(period_mult+3)] = 0
#         for i in train.index:
#             df2["lag_period_{}".format(period_mult+3)][i] = df2.target.shift(24*period_mult+i%24)[i]
#         for i in val.index:
#             df2["lag_period_{}".format(period_mult+3)][i] = df2.target.shift(24*(period_mult+((i-train.index.max())//24))+(i%24))[i]
#         features2.append("lag_period_{}".format(period_mult+3))  
    
    features3 = []
    for period_mult in range(1,3,1):
        df3["lag_period_{}".format(period_mult)] = df3.target.shift(period_mult*lag_period)
        features3.append("lag_period_{}".format(period_mult))
#     for period_mult in range(1,3,1):
#         df3["lag_period_{}".format(period_mult+3)] = df3.target.shift(period_mult*lag_period+24)
#         features3.append("lag_period_{}".format(period_mult+3))
    for period_mult in range(1,l,1):
        df3["lag_period_{}".format(period_mult+3)] = df3.target.shift(lag_period+period_mult)
        features3.append("lag_period_{}".format(period_mult+3))
#     for period_mult in range(1,p,1):
#         df3["lag_period_{}".format(period_mult+3)] = 0
#         for i in train.index:
#             df3["lag_period_{}".format(period_mult+3)][i] = df3.target.shift(24*period_mult+i%24)[i]
#         for i in val.index:
#             df3["lag_period_{}".format(period_mult+3)][i] = df3.target.shift(24*(period_mult+((i-train.index.max())//24))+(i%24))[i]
#         features3.append("lag_period_{}".format(period_mult+3))
    # # лаговые статистики
    
    
    df1['lagf_mean'] = df1[features1[:2]].mean(axis = 1)
    df2['lagf_mean'] = df2[features2[:2]].mean(axis = 1)
    df3['lagf_mean'] = df3[features3[:2]].mean(axis = 1)
    df1['lagf_mean1'] = df1[features1[2:]].mean(axis = 1)
    df2['lagf_mean1'] = df2[features2[2:]].mean(axis = 1)
    df3['lagf_mean1'] = df3[features3[2:]].mean(axis = 1)

    features1.extend(['lagf_mean'])
    features1.extend(['lagf_mean1'])
    model1 = LinearRegression()
    train_df1 = df1[df1.label == 'train'][features1 + ['target']].dropna()
    test_df1 = df1[df1.label == 'val'][features1]

    features2.extend(['lagf_mean'])
    features2.extend(['lagf_mean1'])
    model2 = LinearRegression()
    train_df2 = df2[df2.label == 'train'][features2 + ['target']].dropna()
    test_df2 = df2[df2.label == 'val'][features2]

    features3.extend(['lagf_mean'])
    features3.extend(['lagf_mean1'])
    model3 = LinearRegression()
    train_df3 = df3[df3.label == 'train'][features3 + ['target']].dropna()
    test_df3 = df3[df3.label == 'val'][features3]

    model1.fit(train_df1.drop('target', axis = 1) ,train_df1['target'])
    forecast1 = model1.predict(test_df1)
    if i in x_best:
        forecast1 = forecast1_1
    test_df1['prediction'] = forecast1
    model2.fit(train_df2.drop('target', axis = 1) ,train_df2['target'])
    forecast2 = model2.predict(test_df2)
    if i in y_best:
        forecast2 = forecast2_2
    test_df2['prediction'] = forecast2
    model3.fit(train_df3.drop('target', axis = 1) ,train_df3['target'])
    forecast3 = model3.predict(test_df3)
    if i in z_best:
        forecast3 = forecast3_3
    test_df3['prediction'] = forecast3
    a1 =  np.hstack((np.array(df1.x[:k]),np.array(test_df1.prediction.values)))
    a2 =  np.hstack((np.array(df2.y[:k]),np.array(test_df2.prediction.values)))
    a3 =  np.hstack((np.array(df3.z[:k]),np.array(test_df3.prediction.values)))
    a1 = pd.DataFrame(a1)
    a2= pd.DataFrame(a2)
    a3= pd.DataFrame(a3)
    a1.index = df1.index
    a2.index = df2.index
    a3.index = df3.index
    df['x'] = a1
    df['y'] = a2
    df['z'] = a3
    df['error']  = np.linalg.norm(df[['x', 'y', 'z']].values - df[['x_sim', 'y_sim', 'z_sim']].values, axis=1)
    df_cur = pd.DataFrame(df[k:])
    df_cur = df_cur[['id','error']]
    df_res = df_res.merge(df_cur, how='outer')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
test_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 649462 to 649911
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   lag_period_1   450 non-null    float64
 1   lag_period_2   450 non-null    float64
 2   lag_period_4   450 non-null    float64
 3   lag_period_5   450 non-null    float64
 4   lag_period_6   450 non-null    float64
 5   lag_period_7   450 non-null    float64
 6   lag_period_8   450 non-null    float64
 7   lag_period_9   450 non-null    float64
 8   lag_period_10  450 non-null    float64
 9   lag_period_11  450 non-null    float64
 10  lag_period_12  450 non-null    float64
 11  lag_period_13  450 non-null    float64
 12  lag_period_14  450 non-null    float64
 13  lag_period_15  450 non-null    float64
 14  lag_period_16  450 non-null    float64
 15  lag_period_17  450 non-null    float64
 16  lag_period_18  450 non-null    float64
 17  lag_period_19  450 non-null    float64
 18  la

In [16]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162708 entries, 0 to 162707
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      162708 non-null  int64  
 1   error   162708 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 3.7 MB


In [17]:
df_res.tail()

,id,error
162703,1234089,2412.154939
162704,1234090,2533.676296
162705,1234091,2680.320302
162706,1234092,2849.811239
162707,1234093,3036.067699


In [18]:
df_res[:241]

,id,error
0,718,12096.619003
1,719,12814.879300
2,720,13188.303705
3,721,13114.783726
4,722,12841.122169
...,...,...
236,954,17339.143605
237,955,18543.940803
238,956,19917.172744
239,957,21422.221204


In [19]:
with open("res_123.csv", "w") as f:
    f.write(df_res.to_csv(columns=("id", "error"), index=False))